In [2]:
from objdet.YOLOV8.yolo import *

model = yolo_v8_n()

In [4]:
import torch
x= torch.randn(1,3,640,640)

before_eval = model(x)

model.eval()
after_eval = model(x)

/home/shawn/anaconda3/envs/torch2/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400430266/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
after_eval.shape

torch.Size([1, 84, 8400])

In [10]:
for p in before_eval:
    print(p.shape)

torch.Size([1, 84, 80, 80])
torch.Size([1, 84, 40, 40])
torch.Size([1, 84, 20, 20])


In [23]:
print(before_eval[0][0][0][0])
print(before_eval[1][0][0][0])
print(before_eval[2][0][0][0])

tensor([0.9768, 1.0896, 1.2533, 1.1957, 1.3039, 1.2468, 1.2884, 1.2497, 1.2507,
        1.4928, 1.0433, 1.3386, 1.0496, 1.2606, 0.9307, 0.7794, 0.5714, 0.9071,
        1.6133, 1.4488, 1.5370, 1.7009, 1.0592, 1.5892, 1.2035, 1.1415, 1.5895,
        1.3423, 1.5341, 1.2886, 1.3876, 1.1958, 1.1564, 1.1063, 1.3816, 1.2198,
        1.1402, 1.0430, 0.9471, 1.2430, 1.2217, 1.3149, 1.4910, 1.4101, 0.8597,
        0.8719, 0.9406, 1.1072, 1.1595, 1.1186, 0.9569, 0.9380, 0.8152, 1.1794,
        1.3380, 1.4011, 1.1135, 1.0522, 1.0368, 1.0796, 0.9810, 1.0320, 0.8659,
        1.0675, 1.0642, 1.0022, 1.1772, 1.0927, 1.3046, 1.3729, 1.4304, 1.2504,
        1.0085, 1.1285, 1.0032, 1.2136, 1.2566, 1.1878, 1.0308, 0.9749],
       grad_fn=<SelectBackward0>)
tensor([1.0762, 1.0786, 1.0857, 0.8224, 1.2243, 1.0502, 1.4101, 1.1686, 1.3602,
        1.2024, 0.7550, 1.2048, 1.3575, 1.1047, 1.0687, 1.1626, 1.1720, 1.1908,
        1.1288, 1.0056, 1.2489, 1.2540, 1.0954, 0.9124, 0.9609, 0.9880, 0.9556,
        0.923

In [19]:
after_eval[0][0]

tensor([  4.0065,  12.0005,  20.0014,  ..., 560.1067, 592.1022, 624.1115],
       grad_fn=<SelectBackward0>)

In [11]:
80**2 + 40**2 + 20**2

8400

In [24]:
def non_max_suppression(
    prediction,
    conf_thres=0.25,
    iou_thres=0.45,
    classes=None,
    agnostic=False,
    multi_label=False,
    labels=(),
    max_det=300,
    nc=0,  # number of classes (optional)
    max_time_img=0.05,
    max_nms=30000,
    max_wh=7680,
    in_place=True,
    rotated=False,
):
    """
    Perform non-maximum suppression (NMS) on a set of boxes, with support for masks and multiple labels per box.

    Args:
        prediction (torch.Tensor): A tensor of shape (batch_size, num_classes + 4 + num_masks, num_boxes)
            containing the predicted boxes, classes, and masks. The tensor should be in the format
            output by a model, such as YOLO.
        conf_thres (float): The confidence threshold below which boxes will be filtered out.
            Valid values are between 0.0 and 1.0.
        iou_thres (float): The IoU threshold below which boxes will be filtered out during NMS.
            Valid values are between 0.0 and 1.0.
        classes (List[int]): A list of class indices to consider. If None, all classes will be considered.
        agnostic (bool): If True, the model is agnostic to the number of classes, and all
            classes will be considered as one.
        multi_label (bool): If True, each box may have multiple labels.
        labels (List[List[Union[int, float, torch.Tensor]]]): A list of lists, where each inner
            list contains the apriori labels for a given image. The list should be in the format
            output by a dataloader, with each label being a tuple of (class_index, x1, y1, x2, y2).
        max_det (int): The maximum number of boxes to keep after NMS.
        nc (int, optional): The number of classes output by the model. Any indices after this will be considered masks.
        max_time_img (float): The maximum time (seconds) for processing one image.
        max_nms (int): The maximum number of boxes into torchvision.ops.nms().
        max_wh (int): The maximum box width and height in pixels.
        in_place (bool): If True, the input prediction tensor will be modified in place.

    Returns:
        (List[torch.Tensor]): A list of length batch_size, where each element is a tensor of
            shape (num_boxes, 6 + num_masks) containing the kept boxes, with columns
            (x1, y1, x2, y2, confidence, class, mask1, mask2, ...).
    """
    import torchvision  # scope for faster 'import ultralytics'

    # Checks
    assert 0 <= conf_thres <= 1, f"Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0"
    assert 0 <= iou_thres <= 1, f"Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0"
    if isinstance(prediction, (list, tuple)):  # YOLOv8 model in validation model, output = (inference_out, loss_out)
        prediction = prediction[0]  # select only inference output

    bs = prediction.shape[0]  # batch size
    nc = nc or (prediction.shape[1] - 4)  # number of classes
    nm = prediction.shape[1] - nc - 4
    mi = 4 + nc  # mask start index
    xc = prediction[:, 4:mi].amax(1) > conf_thres  # candidates

    # Settings
    # min_wh = 2  # (pixels) minimum box width and height
    time_limit = 2.0 + max_time_img * bs  # seconds to quit after
    multi_label &= nc > 1  # multiple labels per box (adds 0.5ms/img)

    prediction = prediction.transpose(-1, -2)  # shape(1,84,6300) to shape(1,6300,84)
    if not rotated:
        if in_place:
            prediction[..., :4] = xywh2xyxy(prediction[..., :4])  # xywh to xyxy
        else:
            prediction = torch.cat((xywh2xyxy(prediction[..., :4]), prediction[..., 4:]), dim=-1)  # xywh to xyxy

    t = time.time()
    output = [torch.zeros((0, 6 + nm), device=prediction.device)] * bs
    for xi, x in enumerate(prediction):  # image index, image inference
        # Apply constraints
        # x[((x[:, 2:4] < min_wh) | (x[:, 2:4] > max_wh)).any(1), 4] = 0  # width-height
        x = x[xc[xi]]  # confidence

        # Cat apriori labels if autolabelling
        if labels and len(labels[xi]) and not rotated:
            lb = labels[xi]
            v = torch.zeros((len(lb), nc + nm + 4), device=x.device)
            v[:, :4] = xywh2xyxy(lb[:, 1:5])  # box
            v[range(len(lb)), lb[:, 0].long() + 4] = 1.0  # cls
            x = torch.cat((x, v), 0)

        # If none remain process next image
        if not x.shape[0]:
            continue

        # Detections matrix nx6 (xyxy, conf, cls)
        box, cls, mask = x.split((4, nc, nm), 1)

        if multi_label:
            i, j = torch.where(cls > conf_thres)
            x = torch.cat((box[i], x[i, 4 + j, None], j[:, None].float(), mask[i]), 1)
        else:  # best class only
            conf, j = cls.max(1, keepdim=True)
            x = torch.cat((box, conf, j.float(), mask), 1)[conf.view(-1) > conf_thres]

        # Filter by class
        if classes is not None:
            x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

        # Check shape
        n = x.shape[0]  # number of boxes
        if not n:  # no boxes
            continue
        if n > max_nms:  # excess boxes
            x = x[x[:, 4].argsort(descending=True)[:max_nms]]  # sort by confidence and remove excess boxes

        # Batched NMS
        c = x[:, 5:6] * (0 if agnostic else max_wh)  # classes
        scores = x[:, 4]  # scores
        if rotated:
            boxes = torch.cat((x[:, :2] + c, x[:, 2:4], x[:, -1:]), dim=-1)  # xywhr
            i = nms_rotated(boxes, scores, iou_thres)
        else:
            boxes = x[:, :4] + c  # boxes (offset by class)
            i = torchvision.ops.nms(boxes, scores, iou_thres)  # NMS
        i = i[:max_det]  # limit detections

        # # Experimental
        # merge = False  # use merge-NMS
        # if merge and (1 < n < 3E3):  # Merge NMS (boxes merged using weighted mean)
        #     # Update boxes as boxes(i,4) = weights(i,n) * boxes(n,4)
        #     from .metrics import box_iou
        #     iou = box_iou(boxes[i], boxes) > iou_thres  # IoU matrix
        #     weights = iou * scores[None]  # box weights
        #     x[i, :4] = torch.mm(weights, x[:, :4]).float() / weights.sum(1, keepdim=True)  # merged boxes
        #     redundant = True  # require redundant detections
        #     if redundant:
        #         i = i[iou.sum(1) > 1]  # require redundancy

        output[xi] = x[i]
        if (time.time() - t) > time_limit:
            LOGGER.warning(f"WARNING ⚠️ NMS time limit {time_limit:.3f}s exceeded")
            break  # time limit exceeded

    return output

In [26]:
def xywh2xyxy(x):
    """
    Convert bounding box coordinates from (x, y, width, height) format to (x1, y1, x2, y2) format where (x1, y1) is the
    top-left corner and (x2, y2) is the bottom-right corner.

    Args:
        x (np.ndarray | torch.Tensor): The input bounding box coordinates in (x, y, width, height) format.

    Returns:
        y (np.ndarray | torch.Tensor): The bounding box coordinates in (x1, y1, x2, y2) format.
    """
    assert x.shape[-1] == 4, f"input shape last dimension expected 4 but input shape is {x.shape}"
    y = torch.empty_like(x) if isinstance(x, torch.Tensor) else np.empty_like(x)  # faster than clone/copy
    dw = x[..., 2] / 2  # half-width
    dh = x[..., 3] / 2  # half-height
    y[..., 0] = x[..., 0] - dw  # top left x
    y[..., 1] = x[..., 1] - dh  # top left y
    y[..., 2] = x[..., 0] + dw  # bottom right x
    y[..., 3] = x[..., 1] + dh  # bottom right y
    return y

In [114]:
x = torch.tensor([
    [50, 50, 20, 20],  # 바운딩 박스 1
    [30, 30, 10, 10]   # 바운딩 박스 2
])
x[...]

tensor([[50, 50, 20, 20],
        [30, 30, 10, 10]])

In [28]:
import time
non_max_suppression(after_eval)

[tensor([], size=(0, 6))]

In [106]:
after_eval.shape
a= after_eval.transpose(-1,-2)
a.shape

a

tensor([[[-9.9736e+00, -9.9799e+00,  2.0175e+00,  ...,  9.8039e-06,  9.7436e-06,  9.7837e-06],
         [-5.9689e+00, -9.9772e+00,  1.4011e+01,  ...,  9.8075e-06,  9.7336e-06,  9.7902e-06],
         [-1.9691e+00, -9.9771e+00,  2.6012e+01,  ...,  9.8068e-06,  9.7336e-06,  9.7897e-06],
         ...,
         [ 2.3232e+02,  2.6400e+02,  8.2425e+02,  ...,  1.5653e-04,  1.5539e-04,  1.5598e-04],
         [ 2.4831e+02,  2.6400e+02,  8.7224e+02,  ...,  1.5652e-04,  1.5541e-04,  1.5601e-04],
         [ 2.6433e+02,  2.6400e+02,  9.2026e+02,  ...,  1.5643e-04,  1.5532e-04,  1.5603e-04]]], grad_fn=<TransposeBackward0>)

In [108]:
a[..., :4].shape

torch.Size([1, 8400, 4])

In [ ]:
state_dict = torch.load('/mnt/d/projects/objdet/src/objdet/YOLOV8/yolov8n.pt')

state_dict['model'].state_dict()

In [ ]:
import torch

# YOLOv8 모델 초기화
model = yolo_v8_n()

# 가중치 파일 경로
weight_path = 'yolov8n.pt'

# 가중치 로드
checkpoint = torch.load(weight_path)

# 불필요한 키 제거
if 'model' in checkpoint:
    state_dict = checkpoint['model'].state_dict()
else:
    state_dict = checkpoint

# 필요한 키만 남기기
filtered_state_dict = {}
model_keys = model.state_dict().keys()
for k, v in state_dict.items():
    if k in model_keys:
        filtered_state_dict[k] = v

# 가중치를 모델에 로드
model.load_state_dict(filtered_state_dict, strict=False)

# 모델을 평가 모드로 전환
model.eval()

# # 모델 테스트 (예시 입력)
# x = torch.randn(1, 3, 640, 640)
# output = model(x)
# print(output.shape)

# # 가중치 숫자 값 확인
# for name, param in filtered_state_dict.items():
#     print(f"{name}: {param.shape}")


In [66]:
import cv2
image_path = '/mnt/d/projects/objdet/src/objdet/samples/sample.png'
import numpy as np 

img = cv2.imread(image_path)
H,W,C = img.shape
img = img.astype(np.float32)[:,:,::-1]
img = img/255
img = img.transpose((2,0,1))
img = torch.from_numpy(img).half().unsqueeze(0).cuda()

In [ ]:
m = checkpoint['model']
m.cuda()
m.eval()

In [ ]:
output = m(img)
output[1][0]

In [73]:
output[1][0].shape

torch.Size([1, 144, 72, 96])

In [74]:
a = output[1][0].reshape(1,144,72*96)

In [75]:
a.shape

torch.Size([1, 144, 6912])

In [76]:
non_max_suppression(a)

[tensor([], device='cuda:0', size=(0, 6))]

In [81]:
boxes = non_max_suppression(output, conf_thres = 0.1, iou_thres=0.5)[0].cpu().numpy()

In [85]:
boxes

array([[    -227.25,      -393.5,     -107.75,      -197.5,     0.92627,          16],
       [     -361.5,     -288.25,      -453.5,     -235.75,      0.8623,           1],
       [       -384,      -288.5,        -343,      -234.5,     0.81982,           1],
       [    -380.75,     -289.75,     -362.25,     -231.25,     0.77734,           1],
       [     -371.5,     -289.25,      -406.5,     -234.75,     0.77588,           1],
       [       -548,        -126,      -17.75,     -56.156,     0.76758,           2],
       [    -362.25,        -290,     -452.75,        -239,     0.76367,           1],
       [    -372.25,     -288.25,     -397.75,     -234.75,     0.50586,           1],
       [       -375,      -290.5,        -389,      -236.5,     0.34692,           1],
       [       -376,        -289,        -379,        -237,     0.33105,           1],
       [    -194.75,        -270,      -68.25,        -150,     0.17554,           1],
       [    -51.594,     -155.62,     -72.1

In [96]:
W

768

In [99]:
print(output)
print(boxes)

[tensor([[1.3150e+02, 2.2100e+02, 3.1100e+02, 5.4150e+02, 9.2627e-01, 1.6000e+01],
        [3.3000e+01, 1.0475e+02, 5.6550e+02, 4.1925e+02, 8.6230e-01, 1.0000e+00],
        [4.6850e+02, 7.5125e+01, 6.8950e+02, 1.7150e+02, 7.6758e-01, 2.0000e+00],
        [1.2775e+02, 1.4250e+02, 2.6050e+02, 3.7500e+02, 1.7554e-01, 1.0000e+00],
        [6.2500e-02, 7.5000e-01, 8.2562e+01, 2.4862e+02, 1.1359e-01, 6.2000e+01]], device='cuda:0')]
[[      131.5         221         311       541.5     0.92627          16]
 [         33      104.75       565.5      419.25      0.8623           1]
 [      468.5      75.125       689.5       171.5     0.76758           2]
 [     127.75       142.5       260.5         375     0.17554           1]
 [     0.0625        0.75      82.562      248.62     0.11359          62]]


In [ ]:
import cv2
import numpy as np
import torch

# 이미지 로드
image_path = '/mnt/d/projects/objdet/src/objdet/samples/sample.png'
img = cv2.imread(image_path)
H, W, C = img.shape

# 모델의 출력 예시 (이미지가 640x640으로 스케일링된 경우를 가정)
output = [
    torch.tensor(
        [
            [1.3150e+02, 2.2100e+02, 3.1100e+02, 5.4150e+02, 9.2627e-01, 1.6000e+01],
            [3.3000e+01, 1.0475e+02, 5.6550e+02, 4.1925e+02, 8.6230e-01, 1.0000e+00],
            [4.6850e+02, 7.5125e+01, 6.8950e+02, 1.7150e+02, 7.6758e-01, 2.0000e+00],
            [1.2775e+02, 1.4250e+02, 2.6050e+02, 3.7500e+02, 1.7554e-01, 1.0000e+00],
            [6.2500e-02, 7.5000e-01, 8.2562e+01, 2.4862e+02, 1.1359e-01, 6.2000e+01],
        ],
        device='cuda:0'
    )
]

# Convert the output to CPU and numpy
boxes = output[0].cpu().numpy()

# 이미지 스케일을 고려하여 박스 좌표를 변환
img_scale = 640  # 모델 입력 이미지 크기 (예시로 640을 사용)
scale_w = W / img_scale
scale_h = H / img_scale

# Draw boxes on the original image
for box in boxes:
    x1, y1, x2, y2, conf, cls = box
    x1, y1, x2 ,y2 = int(x1), int(y1), int(x2), int(y2)
    # x1, y1, x2, y2 = int(x1 * W), int(y1 * H), int(x2 * W), int(y2 * H)

    # Check if the box coordinates are valid
    print(f'Drawing box: ({x1}, {y1}, {x2}, {y2}) with confidence: {conf}, class: {cls}')
    
    # Draw rectangle
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Put label
    label = f'{int(cls)}: {conf:.2f}'
    cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image
cv2.imshow('Image with Boxes', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO

m = YOLO('yolov8n.pt')
image_path = '/mnt/d/projects/objdet/src/objdet/samples/sample.png'

m.predict(image_path,save=True)

In [ ]:
img = cv2.imread(image_path)

# Draw boxes on the original image
for box in boxes:
    x1, y1, x2, y2, conf, cls = box
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    
    # Draw rectangle
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Put label
    label = f'{int(cls)}: {conf:.2f}'
    cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image
cv2.imshow('Image with Boxes', img)
cv2.waitKey(0)
cv2.destroyAllWindows()